# Amazon SageMaker AI MLOps: from idea to production in six steps

This sequence of six notebooks takes you from developing your ML idea in a simple notebook to a production solution with automated model building and CI/CD deployment pipelines, and model monitoring.

Follow these steps one by one:
**From idea to production in six steps:**
||||
|---|---|---|
|1. |Experiment in a notebook ||
|2. |Scale with SageMaker AI processing jobs and SageMaker SDK ||
|3. |Operationalize with ML pipeline, model registry, and feature store ||
|4. |Add a model building CI/CD pipeline ||
|5. |Add a model deployment pipeline ||
|6. |Add model and data monitoring ||

There are also additional hands-on examples of other SageMaker features and ML topics, like [A/B testing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html), custom [processing](https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html), [training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) and [inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) containers, [debugging and profiling](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html), [security](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html), [multi-model](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) and [multi-container](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html) endpoints, and [serial inference pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). Explore the notebooks in the folder `additional-topics` to test out these features.

To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

## Star GitHub repository

In [ ]:
%%html

<a class="github-button" href="https://github.com/aws-samples/amazon-sagemaker-from-idea-to-production" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Amazon SageMaker secure MLOps on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

### Click this button ^^^ above ^^^

## Setup
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 The workshop and all notebooks were tested on the SageMaker Distribution Image version <strong>3.6.1</strong>. If you encounter any incompatibility issues, you can stop the JupyterLab space and re-start it with a specific version of the SageMaker Distribution Image.
</div>

![](img/jupyterlab-app-image.png)

In [ ]:
%pip install --upgrade pip "sagemaker<3" boto3

In [ ]:
%pip install --upgrade "mlflow>=2,<3" sagemaker-mlflow

In [ ]:
%pip install -q "sagemaker[local]<3"

### Import packages

In [ ]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep
import mlflow

(sagemaker.__version__,boto3.__version__, mlflow.__version__)

### Set constants

In [ ]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "from-idea-to-prod/xgboost"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()
dataset_file_local_path = "data/bank-additional/bank-additional-full.csv"

initialized = True

print(sm_role)

In [ ]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized
%store dataset_file_local_path

### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value. The following code demonstrates how to access the notebook metadata file and get the `domain_id`.

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

%store domain_id
%store space_name
%store user_profile_name

### Get space details
Use SageMaker API `DescribeSpace` call to get space resource specifications.

In [ ]:
r = boto3.client('sagemaker').describe_space(DomainId=domain_id, SpaceName=space_name)
resource_spec = r['SpaceSettings']['JupyterLabAppSettings']['DefaultResourceSpec']
sm_image = resource_spec.get('SageMakerImageArn', 'not defined')
sm_image_version = resource_spec.get('SageMakerImageVersionAlias', 'not defined')
print(f"""
SageMaker image: \033[1m{sm_image}\033[0m
SageMaker image version: \033[1m{sm_image_version}\033[0m
""")

In [ ]:
r

In [ ]:
%store sm_image
%store sm_image_version

### Connect to MLflow tracking server
If you're running an AWS-led workshop or used the delivered CloudFormation template to provision your workshop environment, an MLflow server must be up and running. If you don't have an MLflow server, follow the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) or run the following code cell to create a new one.

To create and manage an MLflow tracking server and to work with managed MLflow experiements, you need the following permissions attached to the SageMaker execution role:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

Execute the following code to check if you have a running MLflow server.

In [ ]:
# Find an active MLflow server in the account
tracking_servers = [s['TrackingServerArn'] for s 
                    in boto3.client("sagemaker").list_mlflow_tracking_servers()['TrackingServerSummaries']
                    if s['IsActive'] == 'Active']

if len(tracking_servers) < 1:
    print("You don't have any active MLflow servers. Trying to find a server in the status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_arn = tracking_servers[0]
    mlflow_name = tracking_servers[0].split('/')[1]
    print(f"You have {len(tracking_servers)} running MLflow server(s). Get the first server ARN:{mlflow_arn}")

In [ ]:
# This code cell creates a new MLflow server
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{domain_id}-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Server creation request succeded. The server {mlflow_arn} is being created.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
Creation of an MLflow server can take up to 25 minutes. You don't need to wait - proceed with the flow of the workshop.
</div>

In [ ]:
(mlflow_arn, mlflow_name, mlflow.__version__)

In [ ]:
%store mlflow_arn
%store mlflow_name

## Install Docker to enable Studio local mode
Amazon SageMaker Studio applications support the use of local mode to create estimators, processors, and pipelines, then deploy them to a local environment. With local mode, you can test machine learning scripts before running them in Amazon SageMaker managed training or hosting environments. Refer to [Local mode support in Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html) to understand which docker operations the Studio currently supports.

To use local mode in Studio applications, you must install Docker into your JupyterLab space. 

### Check if docker access is enabled

In [ ]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id).get('DomainSettings', {}).get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
If the previous code cell raised an exeption that the docker access is not enabled, you need to enable the access. See the following instructions how to do it.
</div>

In [ ]:
print(f"Domain id: {domain_id}")

### Enable docker access for the SageMaker domain

<div class="alert alert-info">You only need to execute this section if the docker access is not enabled in the domain.
</div>

You need `sagemaker:UpdateDomain` permission in the execution role. You cannot update domain from this notebook because the notebook execution role doesn't have this permission. To update domain settings, you can use one of the following options.

#### Option 1: run `update_domain` in the notebook
If you have the corresponding permissions in the notebook execution role, you can run the following code in a notebook:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
        }
    }
)
```

#### Option 2: run `aws sagemaker` CLI in the  terminal
Make sure you run `AWS CLI` in the terminal where you have the corresponding permissions `sagemaker:UpdateDomain`. Run the following command:

```
aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update DockerSettings={EnableDockerAccess='ENABLED'}
```

For example, you can run the command above in the [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/) in your AWS account.

In [ ]:
# check the updated settings
sm_client.describe_domain(DomainId=domain_id)['DomainSettings']

### Install Docker

In [ ]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version
# validate the Docker Compose plugin installed
docker compose version

# Install Compose Switch. this is a replacement to the Compose V1 docker-compose (python) executable.
# It translates the command line into Compose V2 docker compose then run the latter.
# this is needed by the sagemaker python sdk to run in local mode
# see https://github.com/docker/compose-switch
curl -fL https://raw.githubusercontent.com/docker/compose-switch/master/install_on_linux.sh | sudo sh
docker-compose version

## Data

This example uses the [direct marketing dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing) from UCI's ML Repository:
> [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

Download and unzip the dataset:

In [ ]:
!wget -P data/ -N https://archive.ics.uci.edu/static/public/222/bank+marketing.zip --no-check-certificate

In [ ]:
import zipfile

with zipfile.ZipFile("data/bank+marketing.zip", "r") as z:
    print("Unzipping bank+marketing...")
    z.extractall("data")

with zipfile.ZipFile("data/bank-additional.zip", "r") as z:
    print("Unzipping bank-additional...")
    z.extractall("data")

print("Done")

### See the data

In [ ]:
df_data = pd.read_csv(dataset_file_local_path, sep=";")

pd.set_option("display.max_columns", 500)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

### Upload data to S3

In [ ]:
input_s3_url = sagemaker.Session().upload_data(
    path=dataset_file_local_path,
    bucket=bucket_name,
    key_prefix=f"{bucket_prefix}/input"
)
print(f"Upload the dataset to {input_s3_url}")

%store input_s3_url

## Restart kernel

In [ ]:
# Restart kernel to get the packages
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## Further workshop flow
You can continue the workshop by going through each of the following notebooks in the direct order, e.g. 1-2-3-4....

If you're interested in a particular topic, you can execute some notebooks standalone, as shown in the following flow diagram:

![](img/workshop-flow.png)

Start with the step 1 [Idea Development](01-idea-development.ipynb) or step 3 [SageMaker Pipelines](03-sagemaker-pipeline.ipynb).

## Additional resources

### Documentation
- [Use Amazon SageMaker Built-in Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html)

### Hands-on examples
- [Get started with Amazon SageMaker](https://aws.amazon.com/sagemaker/getting-started/)


### Workshops
- [Amazon SageMaker 101 Workshop](https://catalog.us-east-1.prod.workshops.aws/workshops/0c6b8a23-b837-4e0f-b2e2-4a3ffd7d645b/en-US)